In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.neural_network import MLPClassifier
import time
import datetime
from sklearn.preprocessing import LabelEncoder

In [24]:
data = pd.read_csv('COVID-19_Case_Surveillance_Public_Use_Data.csv')

/home/noman/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
data.head

<bound method NDFrame.head of                   cdc_case_earliest_dt  cdc_report_dt pos_spec_dt    onset_dt  \
0                            2020/10/23    2021/01/28  2020/10/23         NaN   
1                            2020/10/23    2020/10/23  2020/10/23         NaN   
2                            2020/10/23    2020/10/25  2020/10/23  2020/10/23   
3                            2020/10/23    2020/10/25  2020/10/23         NaN   
4                            2020/10/23    2020/10/26  2020/10/23         NaN   
...                                 ...           ...         ...         ...   
6756715                      2021/03/08    2021/03/23         NaN  2021/03/08   
6756716                  "error" : true           NaN         NaN         NaN   
6756717    "message" : "Internal error"           NaN         NaN         NaN   
6756718                  "status" : 500           NaN         NaN         NaN   
6756719                               }           NaN         NaN         NaN  

In [26]:
data.columns

Index(['cdc_case_earliest_dt ', 'cdc_report_dt', 'pos_spec_dt', 'onset_dt',
       'current_status', 'sex', 'age_group', 'race_ethnicity_combined',
       'hosp_yn', 'icu_yn', 'death_yn', 'medcond_yn'],
      dtype='object')

In [27]:
data = data.dropna(how='any')
data['cdc_case_earliest_dt '] = data['cdc_case_earliest_dt '].apply(lambda x:time.mktime(datetime.datetime.strptime(x, "%Y/%m/%d").timetuple())) 
data['cdc_report_dt'] = data['cdc_report_dt'].apply(lambda x:time.mktime(datetime.datetime.strptime(x, "%Y/%m/%d").timetuple())) 
data['pos_spec_dt'] = data['pos_spec_dt'].apply(lambda x:time.mktime(datetime.datetime.strptime(x, "%Y/%m/%d").timetuple())) 
data['onset_dt'] = data['onset_dt'].apply(lambda x:time.mktime(datetime.datetime.strptime(x, "%Y/%m/%d").timetuple())) 

data

<ipython-input-27-40d25733020f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cdc_case_earliest_dt '] = data['cdc_case_earliest_dt '].apply(lambda x:time.mktime(datetime.datetime.strptime(x, "%Y/%m/%d").timetuple()))
<ipython-input-27-40d25733020f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cdc_report_dt'] = data['cdc_report_dt'].apply(lambda x:time.mktime(datetime.datetime.strptime(x, "%Y/%m/%d").timetuple()))
<ipython-input-27-40d25733020f>:4: SettingWithCopyWarning: 
A value is tryin

,cdc_case_earliest_dt,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,race_ethnicity_combined,hosp_yn,icu_yn,death_yn,medcond_yn
2,1.603393e+09,1.603566e+09,1.603393e+09,1.603393e+09,Laboratory-confirmed case,Female,0 - 9 Years,"Black, Non-Hispanic",No,Missing,Missing,Missing
5,1.603307e+09,1.603566e+09,1.603393e+09,1.603307e+09,Laboratory-confirmed case,Female,0 - 9 Years,"Black, Non-Hispanic",No,Missing,No,Missing
7,1.603220e+09,1.603566e+09,1.603393e+09,1.603220e+09,Laboratory-confirmed case,Female,0 - 9 Years,"Black, Non-Hispanic",No,Missing,Missing,Missing
12,1.603307e+09,1.603307e+09,1.603393e+09,1.603307e+09,Laboratory-confirmed case,Female,0 - 9 Years,"Black, Non-Hispanic",No,Unknown,No,No
21,1.603307e+09,1.603307e+09,1.603393e+09,1.603307e+09,Laboratory-confirmed case,Female,0 - 9 Years,"Black, Non-Hispanic",No,Unknown,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...
6439957,1.613502e+09,1.613502e+09,1.616785e+09,1.613502e+09,Laboratory-confirmed case,Female,60 - 69 Years,"White, Non-Hispanic",Missing,Missing,Missing,Missing
6439958,1.612897e+09,1.612897e+09,1.616785e+09,1.612897e+09,Laboratory-confirmed case,Female,60 - 69 Years,"White, Non-Hispanic",No,Missing,Missing,Missing
6439959,1.615835e+09,1.617044e+09,1.616785e+09,1.615835e+09,Probable Case,Female,60 - 69 Years,"White, Non-Hispanic",Missing,Missing,No,No
6439960,1.615748e+09,1.617044e+09,1.616785e+09,1.615748e+09,Laboratory-confirmed case,Male,60 - 69 Years,"White, Non-Hispanic",Yes,No,No,Yes


In [32]:
data['sex'] = LabelEncoder().fit_transform(data['sex'])
data['race_ethnicity_combined'] = LabelEncoder().fit_transform(data['race_ethnicity_combined'])
data['age_group'] = LabelEncoder().fit_transform(data['age_group'])
data['hosp_yn'] = LabelEncoder().fit_transform(data['hosp_yn'])
data['icu_yn'] = LabelEncoder().fit_transform(data['icu_yn'])
data['death_yn'] = LabelEncoder().fit_transform(data['death_yn'])
data['medcond_yn'] = LabelEncoder().fit_transform(data['medcond_yn'])

<ipython-input-32-2f2e01ca27f2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sex'] = LabelEncoder().fit_transform(data['sex'])
<ipython-input-32-2f2e01ca27f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['race_ethnicity_combined'] = LabelEncoder().fit_transform(data['race_ethnicity_combined'])
<ipython-input-32-2f2e01ca27f2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [33]:
Y = data['current_status']
X = data.drop('current_status',axis=1)

In [34]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)


In [35]:
mlp = MLPClassifier(activation='relu',hidden_layer_sizes=(32,64,128))
mlp.fit(X_train,y_train)
prediction = mlp.predict(X_test)


/home/noman/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'                           precision    recall  f1-score   support\n\nLaboratory-confirmed case       0.94      1.00      0.97    167483\n            Probable Case       0.00      0.00      0.00     11533\n\n                 accuracy                           0.94    179016\n                macro avg       0.47      0.50      0.48    179016\n             weighted avg       0.88      0.94      0.90    179016\n'

In [36]:
print(classification_report(y_test,prediction))

                           precision    recall  f1-score   support

Laboratory-confirmed case       0.94      1.00      0.97    167483
            Probable Case       0.00      0.00      0.00     11533

                 accuracy                           0.94    179016
                macro avg       0.47      0.50      0.48    179016
             weighted avg       0.88      0.94      0.90    179016



In [38]:
accuracy_score(y_test,prediction)

0.935575591008625

In [40]:
confusion_matrix(y_test,prediction)

array([[167483,      0],
       [ 11533,      0]])